In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq glpk-utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.9 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 128208 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

In [2]:
import os
import datetime 
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
client = gspread.authorize(creds)
import locale
import calendar
import math



import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import DistanceMetric


In [3]:

tu= gc.open_by_url('https://docs.google.com/spreadsheets/d/1AMjOArJbsAqBnDM8imwYOmjWJMuQw1wFa_ZItR7mJBM/edit#gid=1641204152')
meta= tu.worksheet('PSF CENTRO')
dados = meta.get_all_records()
dados = pd.DataFrame(dados)

In [4]:
dados['Capacidade'] = 4000 

In [5]:
dados.drop(0,inplace = True)

In [6]:
dados.drop(['COORDENADAS'],axis = 1, inplace = True)

In [7]:
dados.shape

(45, 48)

In [8]:
for i in dados.columns[1:47].tolist():
  dados[i]=pd.to_numeric(dados[i])

In [9]:
Ci = list(dados.População)
Kj = list(dados.Capacidade)

In [10]:
modelo = pyo.ConcreteModel()

In [11]:
dij=dados[['1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45']].to_numpy()

In [12]:
modelo.M= range(len(dij))

In [13]:
modelo.N = range(len(dij))

In [14]:
modelo.d = pyo.Param(modelo.M, modelo.N, initialize=lambda modelo, i, j: dij[i][j])
modelo.C = pyo.Param(modelo.M, initialize=lambda modelo, i: Ci[i])
modelo.K = pyo.Param(modelo.N, initialize=lambda modelo, j: Kj[j])

In [15]:
p = 9

In [16]:
modelo.y = pyo.Var(modelo.N, within=pyo.Binary)
modelo.x = pyo.Var(modelo.M, modelo.N, within=pyo.Binary)

In [17]:
def f_obj(modelo):
    return sum(modelo.C[i] * modelo.x[i,j]*modelo.d[i,j] for i in modelo.M for j in modelo.N)

modelo.objetivo = pyo.Objective(rule=f_obj, sense=pyo.minimize)

In [18]:
modelo.restricao_a = pyo.Constraint(expr=sum(modelo.y[j] for j in modelo.N) == p)


In [19]:
modelo.restricao_b = pyo.ConstraintList()
for i in modelo.M:
    modelo.restricao_b.add(sum(modelo.x[i,j] for j in modelo.N) == 1)

In [20]:
modelo.restricao_c = pyo.ConstraintList()
for j in modelo.N:
    modelo.restricao_c.add(sum(modelo.C[i] * modelo.x[i,j] for i in modelo.M) <= modelo.K[j] * modelo.y[j])

In [21]:
resultado=pyo.SolverFactory('glpk', executable='/usr/bin/glpsol').solve(modelo)

In [22]:
# Medianas
list_y = list(modelo.y.keys())
dados_modelo = dados.copy()
dados_modelo['Mediana'] = [modelo.y[i]() for i in list_y]
# Alocações:
list_x = list(modelo.x.keys())
alocacoes = [i for i in list_x if modelo.x[i]() == 1]
alocacoes.sort(key=lambda x:x[0])
medianas = [alocacao[1] for alocacao in alocacoes]
dados_modelo['Alocacao'] = medianas

# Distância
dados_modelo['Distancia'] = [dij[alocacao[0], alocacao[1]] for alocacao in alocacoes]

In [23]:
dados_modelo

,Indices,População,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,Capacidade,Mediana,Alocacao,Distancia
1,1,942,0,2100,2900,2400,1600,2800,2100,750,...,55000,42200,15300,14700,24700,24600,4000,0.0,7,750
2,2,465,2100,0,800,1400,1100,2300,3400,2300,...,56100,45000,13500,12900,22900,24000,4000,0.0,2,800
3,3,847,2900,800,0,1600,1400,2500,3600,2400,...,56600,45400,13000,12400,22400,24200,4000,1.0,2,0
4,4,889,2400,1400,1600,0,450,1600,2800,1600,...,56400,45200,14000,13400,23400,22900,4000,1.0,3,0
5,5,709,1600,1100,1400,450,0,1300,2400,1200,...,55900,44700,13800,13200,23200,22900,4000,0.0,3,450
6,6,1144,2800,2300,2500,1600,1300,0,2200,2200,...,56700,45500,14900,14300,24300,24100,4000,0.0,3,1600
7,7,969,2100,3400,3600,2800,2400,3200,0,1300,...,56100,44900,16000,15400,25400,25200,4000,0.0,7,1300
8,8,777,750,2300,2400,1600,1200,2200,1300,0,...,55300,44100,14800,14200,24200,24000,4000,1.0,7,0
9,9,864,650,2300,2600,1800,1400,2200,1500,600,...,54900,43700,15000,14400,24400,24200,4000,0.0,10,450
10,10,862,800,2400,2900,2100,1700,2500,1800,1000,...,54700,43500,15300,14700,24700,24500,4000,0.0,10,300


In [25]:
dados_modelo['Distancia_total'] = dados_modelo['Distancia'] * dados_modelo['População']
print(dados_modelo)

   Indices  População      1      2      3      4      5      6      7      8  \
1        1        942      0   2100   2900   2400   1600   2800   2100    750   
2        2        465   2100      0    800   1400   1100   2300   3400   2300   
3        3        847   2900    800      0   1600   1400   2500   3600   2400   
4        4        889   2400   1400   1600      0    450   1600   2800   1600   
5        5        709   1600   1100   1400    450      0   1300   2400   1200   
6        6       1144   2800   2300   2500   1600   1300      0   2200   2200   
7        7        969   2100   3400   3600   2800   2400   3200      0   1300   
8        8        777    750   2300   2400   1600   1200   2200   1300      0   
9        9        864    650   2300   2600   1800   1400   2200   1500    600   
10      10        862    800   2400   2900   2100   1700   2500   1800   1000   
11      11        605    500   2300   2600   1800   1400   2200   1600    900   
12      12        553   2200

In [29]:
dados_modelo.to_excel('P-Mediana.xlsx')